In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")

In [ ]:
import dspy

from dspy.datasets import HotPotQA
from dspy.evaluate import answer_exact_match
from hotpotqa_agent import hotpotqa_agent

ds = HotPotQA(
    train_size=0, test_size=0, dev_size=10, eval_seed=2025
)  # deterministic 10

/Users/roshankern/Desktop/Github/dspy-lora-agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'hotpot_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
Generating test split: 100%|██████████| 7405/7405 [00:00<00:00, 96049.40 examples/s]
`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'hotpot_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


In [ ]:
# Configure DSPY with a language model
lm = dspy.LM("gemini/gemini-2.5-flash")
dspy.configure(lm=lm, temperature=0.7, cache=False)


def _as_text(x):
    if isinstance(x, (list, tuple)):
        return " / ".join(map(str, x))
    return str(x)


correct, incorrect = [], []
for ex in ds.dev:
    pred = hotpotqa_agent(question=ex["question"])
    row = {
        "q": ex["question"],
        "gold": _as_text(ex["answer"]),
        "pred": _as_text(getattr(pred, "answer", pred)),
    }
    (correct if answer_exact_match(ex, pred) else incorrect).append(row)


def print_table(title, rows):
    print(f"\n{title} ({len(rows)})")
    print("-" * 100)
    print(f"{'QUESTION':<60} | {'GOLD':<18} | PRED")
    print("-" * 100)
    for r in rows:
        q, g, p = r["q"], r["gold"], r["pred"]
        print(f"{q[:60]:<60} | {g[:18]:<18} | {p}")
    print("-" * 100)


print_table("Correct", correct)
print_table("Incorrect", incorrect)


Correct (6)
----------------------------------------------------------------------------------------------------
QUESTION                                                     | GOLD               | PRED
----------------------------------------------------------------------------------------------------
Pehchaan: The Face of Truth stars Vinod Khanna, Rati Agnihot | Raveena Tandon     | Raveena Tandon
Trim is a transitway station in the east end of a city that  | 934,243            | 934,243
Which French crime drama starred Antonythasan Jesuthasan, wr | Dheepan            | Dheepan
What is the name of the South Korean-Chinese boy group whose | EXO                | EXO
Who was the composer of the 2011 British-American 3D compute | Chris Bacon        | Chris Bacon
Who is credited for The Internship role that corresponds to  | Shawn Levy         | Shawn Levy
----------------------------------------------------------------------------------------------------

Incorrect (4)
------------------

In [9]:
# Configure DSPY with a language model
lm = dspy.LM(
    "openai/llama3.2:3b",  # anything smaller than 3.2:3b is too dumb for DSPY parsing
    api_key="makora_bio_endpoint",
    api_base="https://roshan-kern--ollama-endpoint-ollama-api.modal.run/v1",
)
dspy.configure(lm=lm, temperature=0.7, cache=False)


def _as_text(x):
    if isinstance(x, (list, tuple)):
        return " / ".join(map(str, x))
    return str(x)


correct, incorrect = [], []
for ex in ds.dev:
    pred = hotpotqa_agent(question=ex["question"])
    row = {
        "q": ex["question"],
        "gold": _as_text(ex["answer"]),
        "pred": _as_text(getattr(pred, "answer", pred)),
    }
    (correct if answer_exact_match(ex, pred) else incorrect).append(row)


def print_table(title, rows):
    print(f"\n{title} ({len(rows)})")
    print("-" * 100)
    print(f"{'QUESTION':<60} | {'GOLD':<18} | PRED")
    print("-" * 100)
    for r in rows:
        q, g, p = r["q"], r["gold"], r["pred"]
        print(f"{q[:60]:<60} | {g[:18]:<18} | {p}")
    print("-" * 100)


print_table("Correct", correct)
print_table("Incorrect", incorrect)


Correct (2)
----------------------------------------------------------------------------------------------------
QUESTION                                                     | GOLD               | PRED
----------------------------------------------------------------------------------------------------
Pehchaan: The Face of Truth stars Vinod Khanna, Rati Agnihot | Raveena Tandon     | Raveena Tandon
Who was the composer of the 2011 British-American 3D compute | Chris Bacon        | Chris Bacon
----------------------------------------------------------------------------------------------------

Incorrect (8)
----------------------------------------------------------------------------------------------------
QUESTION                                                     | GOLD               | PRED
----------------------------------------------------------------------------------------------------
What is the name of the person who helped work on Les Sylphi | Michael Fokine     | Fred Astai